In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# ---- 1. Load Excel files ----
df_country = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Country.xlsx")
df_region = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Region.xlsx")
df_continent = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Continent.xlsx")
df_city = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\City.xlsx")
df_user = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\User.xlsx")
df_transaction = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Transaction.xlsx")
df_mode = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Mode.xlsx")
df_attraction = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Type.xlsx")  # this holds attraction type mapping
df_item = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Item.xlsx")# detailed attraction info
df_updated_item = pd.read_excel(r"C:\Users\C Sutharsan\Downloads\GUVI class notes AIML\Capstone_project\Project_4\Final\Datasets\Updated_Item.xlsx")

 
# ---- 2. Hierarchical merges ----
df_region_full = df_region.merge(df_continent, on="ContinentId", how="inner")
df_country_full = df_country.merge(df_region_full, on="RegionId", how="inner")
df_city_full = df_city.merge(df_country_full, on="CountryId", how="inner")
df_user_full = df_user.merge(df_city, on=["CityId", "CountryId"], how="inner")
df_trans_user = df_transaction.merge(df_user_full, on="UserId", how="inner")
df_trans_user_mode = df_trans_user.merge(df_mode, left_on="VisitMode", right_on="VisitModeId", how="inner")
df_item_full = pd.concat([df_item, df_updated_item], ignore_index=True)
df_item_combined = df_item_full.drop_duplicates(subset='AttractionId', keep='last')
df_trans_user_mode = df_trans_user_mode.drop(columns=['VisitMode_x'])
df_attraction_full = df_item_combined.merge(df_attraction, on="AttractionTypeId", how="inner")
df = df_trans_user_mode.merge(df_attraction_full, on="AttractionId", how="inner")

# Save final merged + cleaned dataset ----
df.to_csv("merged_tourism_dataset.csv", index=False)
print("Saved merged dataset to merged_tourism_dataset.csv")


Saved merged dataset to merged_tourism_dataset.csv


In [49]:
# Filling missing User_City_ID with a default value (like -1)
df['User_City_Id'] = df['CityId'].fillna(-1)

# Adding a flag column to indicate originally missing data
df['Missing_City_ID'] = df['CityId'] == -1

# Standardizing city names (removing extra spaces, standardizing case)
df['CityName'] = df['CityName'].str.strip().str.title()

# Resolving categorical variable discrepancies
categorical_cols = ['VisitMode', 'AttractionType', 'Continent', 'Region', 'Country']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].str.strip()

In [50]:
# Removing invalid ratings
print(f"Invalid ratings (outside 1-5): {len(df[(df['Rating'] < 1) | (df['Rating'] > 5)])}")
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]

# Handle outliers in Rating
Q1 = df['Rating'].quantile(0.25)
Q3 = df['Rating'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nRating outliers: {len(df[(df['Rating'] < lower_bound) | (df['Rating'] > upper_bound)])} values")

# Capping outliers
df['Rating'] = df['Rating'].clip(lower=1, upper=5)  # Assuming ratings are 1-5 scale

Invalid ratings (outside 1-5): 0

Rating outliers: 2647 values


### Feature engineering

In [52]:
# Time-based features
df['VisitSeason'] = df['VisitMonth'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})
df['VisitQuarter'] = ((df['VisitMonth'] - 1) // 3) + 1

In [53]:
## Sorting by time to ensure no leakage
df = df.sort_values(['UserId', 'VisitYear', 'VisitMonth'])

# Calculating user features based on previous behavior only
df['user_previous_visits'] = df.groupby('UserId').cumcount()
df['user_avg_rating_before'] = df.groupby('UserId')['Rating'].transform(lambda x: x.shift(1).expanding().mean())
df['user_avg_rating_before'] = df['user_avg_rating_before'].fillna(3.0)  # neutral default
df['user_rating_trend'] = df.groupby('UserId')['Rating'].diff()

In [54]:
# Calculating attraction features based on previous behavior only
def get_previous_stats(group):
    # Calculating previous visits for each attraction
    group = group.reset_index(drop=True)
    group['attraction_previous_visits'] = range(len(group))
    # Calculating rolling average rating excluding current visit  
    group['attraction_avg_rating_before'] = group['Rating'].shift(1).expanding().mean()
    group['attraction_avg_rating_before'] = group['attraction_avg_rating_before'].fillna(3.0)
    # Counting unique previous visitors
    group['attraction_previous_visitors'] = group.groupby('UserId').cumcount().shift(1).max()
    group['attraction_previous_visitors'] = group['attraction_previous_visitors'].fillna(0)
    return group

df = df.groupby('AttractionId').apply(get_previous_stats).reset_index(drop=True)
df

C:\Users\C Sutharsan\AppData\Local\Temp\ipykernel_7208\3174178046.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('AttractionId').apply(get_previous_stats).reset_index(drop=True)


,TransactionId,UserId,VisitYear,VisitMonth,AttractionId,Rating,ContinentId,RegionId,CountryId,CityId,...,User_City_Id,Missing_City_ID,VisitSeason,VisitQuarter,user_previous_visits,user_avg_rating_before,user_rating_trend,attraction_previous_visits,attraction_avg_rating_before,attraction_previous_visitors
0,146773,90,2016,10,369,5,3,12,85,2558.0,...,2558.0,False,Fall,4,2,5.000000,0.0,0,3.000000,2.0
1,144692,98,2017,10,369,3,3,14,106,3551.0,...,3551.0,False,Fall,4,0,3.000000,NaN,1,5.000000,2.0
2,146026,103,2017,4,369,5,1,4,22,106.0,...,106.0,False,Spring,2,2,4.500000,0.0,2,4.000000,2.0
3,147748,149,2016,3,369,3,4,15,109,4616.0,...,4616.0,False,Spring,1,0,3.000000,NaN,3,4.333333,2.0
4,147733,155,2016,3,369,2,3,11,74,2468.0,...,2468.0,False,Spring,1,0,3.000000,NaN,4,4.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43388,211227,87100,2018,9,1297,4,5,21,159,7460.0,...,7460.0,False,Fall,3,3,2.666667,1.0,700,3.547143,10.0
43389,211238,88112,2016,2,1297,5,5,17,133,6164.0,...,6164.0,False,Winter,1,0,3.000000,NaN,701,3.547789,10.0
43390,211239,88112,2016,2,1297,4,5,17,133,6164.0,...,6164.0,False,Winter,1,1,5.000000,-1.0,702,3.549858,10.0
43391,211240,88112,2016,2,1297,4,5,17,133,6164.0,...,6164.0,False,Winter,1,2,4.500000,0.0,703,3.550498,10.0


In [55]:
# City-level features (aggregate stats that don't use target)
city_stats = df.groupby('CityId').size().to_frame('city_popularity')
df = df.merge(city_stats, left_on='CityId', right_index=True, how='left')
df['city_popularity'] = df['city_popularity'].fillna(0)

In [56]:
# Creating interaction features
# Merge to bring in Continent column
df = df.merge(df_continent, on="ContinentId", how="left")
df['user_continent'] = df['UserId'].astype(str) + '_' + df['Continent']
df['user_attraction_type'] = df['UserId'].astype(str) + '_' + df['AttractionTypeId'].astype(str)
df['attraction_type_season'] = df['AttractionTypeId'].astype(str) + '_' + df['VisitSeason']
df = df.merge(df_region, on="RegionId", how = "left")
df = df.merge(df_country, on="CountryId", how = "left")

In [58]:
df = df.drop(columns = ['ContinentId_x', 'RegionId_x'])
df

,TransactionId,UserId,VisitYear,VisitMonth,AttractionId,Rating,CountryId,CityId,CityName,VisitModeId,...,attraction_previous_visitors,city_popularity,Continent,user_continent,user_attraction_type,attraction_type_season,Region,ContinentId_y,Country,RegionId_y
0,146773,90,2016,10,369,5,85,2558.0,Hebron,4,...,2.0,3,Asia,90_Asia,90_13,13_Fall,Middle East,3,Palestine,12
1,144692,98,2017,10,369,3,106,3551.0,Singapore,2,...,2.0,2765,Asia,98_Asia,98_13,13_Fall,South East Asia,3,Singapore,14
2,146026,103,2017,4,369,5,22,106.0,Cape Town,2,...,2.0,114,Africa,103_Africa,103_13,13_Spring,Southern Africa,1,South Africa,4
3,147748,149,2016,3,369,3,109,4616.0,Mandurah,4,...,2.0,74,Australia & Oceania,149_Australia & Oceania,149_13,13_Spring,Australia,4,Australia,15
4,147733,155,2016,3,369,2,74,2468.0,Taipei,4,...,2.0,48,Asia,155_Asia,155_13,13_Spring,East Asia,3,Taiwan,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43388,211227,87100,2018,9,1297,4,159,7460.0,Lyon,2,...,10.0,16,Europe,87100_Europe,87100_44,44_Fall,Western Europe,5,France,21
43389,211238,88112,2016,2,1297,5,133,6164.0,Bratislava Region,2,...,10.0,4,Europe,88112_Europe,88112_44,44_Winter,Central Europe,5,Slovakia,17
43390,211239,88112,2016,2,1297,4,133,6164.0,Bratislava Region,2,...,10.0,4,Europe,88112_Europe,88112_44,44_Winter,Central Europe,5,Slovakia,17
43391,211240,88112,2016,2,1297,4,133,6164.0,Bratislava Region,2,...,10.0,4,Europe,88112_Europe,88112_44,44_Winter,Central Europe,5,Slovakia,17


In [62]:
df.to_csv("Final_dataset.csv", index = False)
print("Saved final dataset to final_dataset.csv")

Saved final dataset to final_dataset.csv
